In [1]:
import jsonlines
from pptx import Presentation

In [2]:
with jsonlines.open('f30k_test.jsonl') as reader:
    f30k_fns = [item['img_path'] for item in reader]
with jsonlines.open('f30k_test.jsonl') as reader:
    f30k_items = [item for item in reader]
with jsonlines.open('coco_test.jsonl') as reader:
    coco_fns = ['COCO_val2014_%012d.jpg' % item['id'] for item in reader]
with jsonlines.open('coco_test.jsonl') as reader:
    coco_items = [{'sentences': item['sentences'], 'id': item['id'], 'img_path': 'COCO_val2014_%012d.jpg' % item['id']} 
                  for item in reader]

test_fns = []
test_items = []
ix = 0
cur_dir = 0
for ii, fn in enumerate(f30k_fns):
    if ix % 100 == 0:
        cur_dir += 1
    imgpath = 'images/%02d/%s' % (cur_dir, fn)
    test_fns.append(imgpath)
    test_items.append(f30k_items[ii])
    ix += 1
for ii, fn in enumerate(coco_fns):
    if ix % 100 == 0:
        cur_dir += 1
    imgpath = 'images/%02d/%s' % (cur_dir, fn)
    test_fns.append(imgpath)
    test_items.append(coco_items[ii])
    ix += 1

few_ids = [s.strip() for s in open('f30k_few_train.txt').readlines()]

print(f30k_fns[:2], f30k_fns[-2:])
print(coco_fns[:2], coco_fns[-2:])
print(few_ids[:2], few_ids[-2:])

['1007129816.jpg', '1009434119.jpg'] ['97233789.jpg', '97234558.jpg']
['COCO_val2014_000000391895.jpg', 'COCO_val2014_000000060623.jpg'] ['COCO_val2014_000000413120.jpg', 'COCO_val2014_000000369771.jpg']
['3497238310', '3240048764'] ['3331797838', '3648081498']


## en

In [8]:
en_te = [{"sentences": [d['sentences'][0]], "id": d['id'], "img_path": d['img_path']} for d in test_items]
with jsonlines.open('annotations/en/test.jsonl', 'w') as writer:
    writer.write_all(en_te)

## de

In [7]:
de_fns = [s.strip() for s in open('annotations/de/train.lst').readlines()]
de_caps = [s.strip() for s in open('annotations/de/train.1.de').readlines()]
de_tr = [{"sentences": [cap], "id": fn.split('.')[0], "img_path": fn} for (cap, fn) in zip(de_caps, de_fns)]

for n in [1, 5, 10, 25, 50, 100]:
    de_few = []
    for e in de_tr:
        if e['id'] in few_ids[:n]:
            de_few.append(e)
    with jsonlines.open(f'annotations/de/train_{n}.jsonl', 'w') as writer:
        writer.write_all(de_few)

In [139]:
de_fns = [s.strip() for s in open('annotations/de/test_2016.lst').readlines()]
de_caps = [s.strip() for s in open('annotations/de/test_2016.1.de').readlines()]
de_te = [{"sentences": [cap], "id": fn.split('.')[0], "img_path": fn} for (cap, fn) in zip(de_caps, de_fns)]

for cur_dir in range(11, 21):
    start_ix = 100*(cur_dir-1)
    prs = Presentation(f'annotations/de/german_{cur_dir}.pptx')
    for ix, slide in enumerate(list(prs.slides)[3:-1]):
        t = list(slide.placeholders)[0].text
        t = " ".join(t.strip().split())
        fn = test_fns[start_ix+ix]
        d = dict(test_items[start_ix+ix])
        d['sentences'] = [t]
        de_te.append(d)

print(len(de_te))
with jsonlines.open('annotations/de/test.jsonl', 'w') as writer:
    writer.write_all(de_te)

2000


### zh

In [10]:
fn = "annotations/zh/flickr8k-cn/flickr8kzhc.caption.txt"  # Human generated!
seg_lines = [l.strip() for l in open(fn).readlines()]
imgids = [seg_line.split()[0].split('#')[0].split("_")[0] for seg_line in seg_lines]
clean_lines = [seg_line.split()[1] for seg_line in seg_lines]

for n in [1, 5, 10, 25, 50, 100]:
    zh_few = []
    cur_imgs = set()
    for img, line in zip(imgids, clean_lines):
        if img in few_ids[:n] and img not in cur_imgs:
            cur_imgs.add(img)
            zh_few.append({"sentences": [line], "id": img, "img_path": img + '.jpg'})
    print(len(zh_few))
    with jsonlines.open(f'annotations/zh/train_{n}.jsonl', 'w') as writer:
        writer.write_all(zh_few)

1
5
10
25
50
100


In [140]:
zh_te = []
for cur_dir in range(1, 21):
    start_ix = 100*(cur_dir-1)
    prs = Presentation('annotations/zh/chinese_%02d.pptx' % cur_dir)
    for ix, slide in enumerate(list(prs.slides)[3:-1]):
        t = list(slide.placeholders)[0].text
        t = " ".join(t.strip().split())
        fn = test_fns[start_ix+ix]
        d = dict(test_items[start_ix+ix])
        d['sentences'] = [t]
        zh_te.append(d)

print(len(zh_te))
with jsonlines.open('annotations/zh/test.jsonl', 'w') as writer:
    writer.write_all(zh_te)

2000


## tr

In [4]:
import json

In [11]:
fn = "annotations/tr/flickr8k-tr/tasviret8k_captions.json"
j = json.load(open(fn))
flickr8k_tr = dict()
for e in j['images']:
    img = e['filename'].split('_')[0]
    flickr8k_tr[img] = []
    for s in e['sentences']:
        flickr8k_tr[img].append(s['raw'])

for n in [1, 5, 10, 25, 50, 100]:
    tr_few = []
    cur_imgs = set()
    for img, lines in flickr8k_tr.items():
        if img in few_ids[:n] and img not in cur_imgs:
            cur_imgs.add(img)
            tr_few.append({"sentences": [lines[0]], "id": img, "img_path": img + '.jpg'})
    print(len(tr_few))
    with jsonlines.open(f'annotations/tr/train_{n}.jsonl', 'w') as writer:
        writer.write_all(tr_few)

1
5
10
25
50
100


In [20]:
tr_te = []
for cur_dir in range(1, 21):
    start_ix = 100*(cur_dir-1)
    prs = Presentation('annotations/tr/turkish_%02d.pptx' % cur_dir)
    for ix, slide in enumerate(list(prs.slides)[3:-1]):
        t = list(slide.placeholders)[0].text
        t = " ".join(t.strip().split())
        fn = test_fns[start_ix+ix]
        d = dict(test_items[start_ix+ix])
        d['sentences'] = [t]
        tr_te.append(d)

print(len(tr_te))
with jsonlines.open('annotations/tr/test.jsonl', 'w') as writer:
    writer.write_all(tr_te)

2000


## ja

In [119]:
from glob import glob

In [12]:
for n in [1, 5, 10, 25, 50, 100]:
    ja_few = []
    for img in few_ids[:n]:
        fn = f"annotations/ja/Flickr30kEnt-JP/Sentences_jp_v2/{img}.txt"
        with open(fn) as f:
            l = f.readlines()[0]
            seg_line = l.strip()
            clean_line = ''.join(e.split(':')[0] for e in seg_line.split()[1:]).replace(']', '')
            ja_few.append({"sentences": [clean_line], "id": img, "img_path": img + '.jpg'})
    print(len(ja_few))
    with jsonlines.open(f'annotations/ja/train_{n}.jsonl', 'w') as writer:
        writer.write_all(ja_few)

1
5
10
25
50
100


In [131]:
j = json.load(open("annotations/ja/STAIR-captions/stair_captions_v1.2_val.json"))
coco_ja = dict()
for e in j['annotations']:
    img = e['image_id']
    sent = e['caption']
    coco_ja[img] = sent

ja_te = []
for item in en_te[:1000]:
    img = item['id']
    fn = f"annotations/ja/Flickr30kEnt-JP/Sentences_jp_v2/{img}.txt"
    with open(fn) as f:
        l = f.readlines()[0]
        seg_line = l.strip()
        clean_line = ''.join(e.split(':')[0] for e in seg_line.split()[1:]).replace(']', '')
        ja_te.append({"sentences": [clean_line], "id": img, "img_path": img + '.jpg'})
for item in en_te[1000:]:
    img = item['id']
    sent = coco_ja[img]
    ja_te.append({"sentences": [sent], "id": img, "img_path": 'COCO_val2014_%012d.jpg' % img})
    
print(len(ja_te))
with jsonlines.open('annotations/ja/test.jsonl', 'w') as writer:
    writer.write_all(ja_te)

2000


## es

In [17]:
for n in [1, 5, 10, 25, 50, 100]:
    es_few = []
    for cur_dir in range(21, 22):
        start_ix = 100*(cur_dir-1)
        prs = Presentation('annotations/es/spanish_%02d.pptx' % cur_dir)
        for ix, slide in enumerate(list(prs.slides)[4:-1]):
            t = list(slide.placeholders)[0].text
            t = " ".join(t.strip().split())
            img = few_ids[ix]
            if img in few_ids[:n]:
                d = {"sentences": [t], "id": img, "img_path": img + '.jpg'}
                es_few.append(d)
    print(len(es_few))
    with jsonlines.open(f'annotations/es/train_{n}.jsonl', 'w') as writer:
        writer.write_all(es_few)

1
5
10
25
50
100


In [149]:
es_te = []
for cur_dir in range(1, 21):
    start_ix = 100*(cur_dir-1)
    prs = Presentation('annotations/es/spanish_%02d.pptx' % cur_dir)
    for ix, slide in enumerate(list(prs.slides)[4:-1]):
        t = list(slide.placeholders)[0].text
        t = " ".join(t.strip().split())
        fn = test_fns[start_ix+ix]
        d = dict(test_items[start_ix+ix])
        d['sentences'] = [t]
        es_te.append(d)

print(len(es_te))
with jsonlines.open('annotations/es/test.jsonl', 'w') as writer:
    writer.write_all(es_te)

2000


## id

In [18]:
for n in [1, 5, 10, 25, 50, 100]:
    id_few = []
    for cur_dir in range(21, 22):
        start_ix = 100*(cur_dir-1)
        prs = Presentation('annotations/id/indonesian_%02d.pptx' % cur_dir)
        for ix, slide in enumerate(list(prs.slides)[3:-1]):
            t = list(slide.placeholders)[0].text
            t = " ".join(t.strip().split())
            img = few_ids[ix]
            if img in few_ids[:n]:
                d = {"sentences": [t], "id": img, "img_path": img + '.jpg'}
                id_few.append(d)
    print(len(id_few))
    with jsonlines.open(f'annotations/id/train_{n}.jsonl', 'w') as writer:
        writer.write_all(id_few)

1
5
10
25
50
100


In [150]:
id_te = []
for cur_dir in range(1, 21):
    start_ix = 100*(cur_dir-1)
    prs = Presentation('annotations/id/indonesian_%02d.pptx' % cur_dir)
    for ix, slide in enumerate(list(prs.slides)[3:-1]):
        t = list(slide.placeholders)[0].text
        t = " ".join(t.strip().split())
        fn = test_fns[start_ix+ix]
        d = dict(test_items[start_ix+ix])
        d['sentences'] = [t]
        id_te.append(d)

print(len(id_te))
with jsonlines.open('annotations/id/test.jsonl', 'w') as writer:
    writer.write_all(id_te)

2000


## ru

In [6]:
for n in [1, 5, 10, 25, 50, 100]:
    ru_few = []
    for cur_dir in range(21, 22):
        start_ix = 100*(cur_dir-1)
        prs = Presentation('annotations/ru/russian_%02d.pptx' % cur_dir)
        for ix, slide in enumerate(list(prs.slides)[4:-1]):
            t = list(slide.placeholders)[0].text
            t = " ".join(t.strip().split())
            img = few_ids[ix]
            if img in few_ids[:n]:
                d = {"sentences": [t], "id": img, "img_path": img + '.jpg'}
                ru_few.append(d)
    print(len(ru_few))
    with jsonlines.open(f'annotations/ru/train_{n}.jsonl', 'w') as writer:
        writer.write_all(ru_few)

1
5
10
25
50
100


In [7]:
ru_te = []
for cur_dir in range(1, 21):
    start_ix = 100*(cur_dir-1)
    prs = Presentation('annotations/ru/russian_%02d.pptx' % cur_dir)
    for ix, slide in enumerate(list(prs.slides)[4:-1]):
        t = list(slide.placeholders)[0].text
        t = " ".join(t.strip().split())
        fn = test_fns[start_ix+ix]
        d = dict(test_items[start_ix+ix])
        d['sentences'] = [t]
        ru_te.append(d)

print(len(ru_te))
with jsonlines.open('annotations/ru/test.jsonl', 'w') as writer:
    writer.write_all(ru_te)

2000
